## Multi-Variate Multi Target PyTorch

#### Load Libraries

In [28]:
import pandas as pd
import os
import pickle

from torchsummary import summary
from models.get_model import get_model

import  pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from ptl_dataloader import MIMODataModule
from ptl_model import MIMOPredictor

In [29]:
# Random Seed Pytorch Lightning
pl.seed_everything(42)

Global seed set to 42


42

In [30]:
# Loading Data and Splitting into features and target arrays
#data = pd.read_csv("mimo_data.csv")
with open('mimo_data.pkl', 'rb') as f:
    data = pickle.load(f)

features = data.values[ : , : 6]
targets = data.values[ : , 6 :]

#### Train-Test Split

In [31]:
split_ratio = 0.8
train_features = features[ : int(len(features) * split_ratio)]
test_features = features[int(len(features) * split_ratio) : ]

train_targets = targets[ : int(len(features) * split_ratio)]
test_targets = targets[int(len(features) * split_ratio) : ]

print("Train Size: ", len(train_features))
print("Tes Size: ", len(test_features))

Train Size:  80000
Tes Size:  20000


In [32]:
# Number of input columns
n_inputs = train_features.shape[1]  # Number of features
n_outputs = train_targets.shape[1]  # Number of targets

print("Number of Inputs: ", n_inputs)
print("Number of Outputs: ", n_outputs)

Number of Inputs:  6
Number of Outputs:  4


#### Getting a Specific Model Architecture

In [33]:
# Define the model
architecture = "lstm"
net = get_model(architecture,n_features=n_inputs,  n_targets=n_outputs)
summary(net, (1, n_inputs))

Layer (type:depth-idx)                   Output Shape              Param #
├─LSTM: 1-1                              [-1, 1, 128]              201,728
├─Linear: 1-2                            [-1, 4]                   516
Total params: 202,244
Trainable params: 202,244
Non-trainable params: 0
Total mult-adds (M): 0.20
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.77
Estimated Total Size (MB): 0.77


Layer (type:depth-idx)                   Output Shape              Param #
├─LSTM: 1-1                              [-1, 1, 128]              201,728
├─Linear: 1-2                            [-1, 4]                   516
Total params: 202,244
Trainable params: 202,244
Non-trainable params: 0
Total mult-adds (M): 0.20
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.77
Estimated Total Size (MB): 0.77

#### Loading Data with ptl data module

In [34]:
BATCH_SIZE = 64
data_module = MIMODataModule(train_features, train_targets, 
      test_features, test_targets, batch_size=BATCH_SIZE)
  
data_module.setup()

In [35]:
# Model
model = MIMOPredictor(net)

#### Defining Callbacks

In [36]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k = 2,
    verbose = True,
    monitor = "val_loss",
    mode = "min"
)

# Log to Tensor Board
logger = TensorBoardLogger("lightning_logs", name = "mimo-predict")

# Stop trainining if model is not improving
early_stopping_callback = EarlyStopping(monitor = "val_loss", patience = 30)

# Progress bar
progress_bar = TQDMProgressBar(refresh_rate=5)


In [37]:
N_EPOCHS = 1
trainer = pl.Trainer(
    logger = logger,
    enable_progress_bar=True,
    callbacks = [early_stopping_callback, early_stopping_callback, progress_bar],
    max_epochs = N_EPOCHS,
    gpus = 1,
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


#### Training the Model

In [38]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | LSTMModel | 202 K 
1 | criterion | MSELoss   | 0     
----------------------------------------
202 K     Trainable params
0         Non-trainable params
202 K     Total params
0.809     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]